In [1]:
import numpy as np

import import_notebook
from readWrite import ReadWrite

importing Jupyter notebook from readWrite.ipynb


In [2]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"
tweetData = "combine_tweets.txt"
tweetLive = "user_tweet_live.txt"
tweetTest = "user_tweets.txt"
tweetTestResult = "docVectors1Result.txt"

readWrite = ReadWrite()
tdata = readWrite.readOriFile(tweetData)
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

livedata = readWrite.readOriFile(tweetLive)
livedata = np.array(livedata)

testdata = readWrite.readOriFile(tweetTest)
test_resultdata = readWrite.readFileClassifier(tweetTestResult)
testdata = np.array(testdata)
test_resultdata = np.array(test_resultdata)

tdata = np.array(tdata)
xdata = np.array(xdata)
ydata = np.array(ydata)

print("live data: {}".format(livedata.shape))
print("test data: {}".format(testdata.shape))
print("test_result data: {}".format(test_resultdata.shape))

print("tdata: {}".format(tdata.shape))
print("xdata: {}".format(xdata.shape))
print("ydata: {}".format(ydata.shape))

line size more than 20, trim to: ['2356', ' 1257', ' 1689', ' 1391', ' 2330', ' 791', ' 80', ' 1060', ' 2002', ' 1884', ' 1060', ' 300', ' 2330', ' 1135', ' 1799', ' 2073', ' 1165', ' 2253', ' 185', ' 1207']
line size more than 20, trim to: ['2356', ' 1257', ' 1239', ' 500', ' 851', ' 1568', ' 1932', ' 851', ' 1239', ' 2153', ' 138', ' 1239', ' 455', ' 1008', ' 1044', ' 21', ' 1243', ' 1302', ' 851', ' 1503']
Read complete, total records: 1814
Read complete, total records: 1814
Read complete, total records: 731
live data: (340,)
test data: (731,)
test_result data: (731,)
tdata: (1814,)
xdata: (1814, 20)
ydata: (1814,)


In [3]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(tdata)
print("X_train_count: {}".format(X_train_count.shape))

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
print("X_train_tfidf: {}".format(X_train_tfidf.shape))

# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, ydata)

X_train_count: (1814, 3473)
X_train_tfidf: (1814, 3473)


In [4]:
'''
Naive Bayes
'''
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# The names ‘vect’ , ‘tfidf’ and ‘clf’ are arbitrary but will be used later.
# We will be using the 'text_clf' going forward.
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(tdata, ydata)

# Performance of NB Classifier
predicted = text_clf.predict(testdata)
print("Naive Bayes")
print("Acuraccy: %s" % np.mean(predicted == test_resultdata))

Naive Bayes
Acuraccy: 0.812585499316


In [5]:
'''
SVM
'''
# Training Support Vector Machines - SVM and calculating its performance
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])

text_clf_svm = text_clf_svm.fit(tdata, ydata)
predicted_svm = text_clf_svm.predict(testdata)
print("SVM")
print("Acuraccy: %s" % np.mean(predicted_svm == test_resultdata))

SVM
Acuraccy: 0.893296853625


In [6]:
'''
Grid Search
'''
# Here, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name (remember the arbitrary name we gave). 
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(tdata, ydata)
predicted_gs = gs_clf.predict(testdata)
print("Grid Search")
print("Acuraccy: %s" % np.mean(predicted_gs == test_resultdata))

Grid Search
Acuraccy: 0.987688098495


In [32]:
def printPredict(predicted, data):
    for idx, score in enumerate(predicted):    
        if(score > 0): 
            print(score)
            print(data[idx])
            
# Testing on unknown/live data
predicted = gs_clf.predict(livedata)
# printPredict(predicted, livedata)

sentence = []
sentence.append("@SBSTransit_Ltd Is there a delay on the NE Line now???")
sen_predicted = gs_clf.predict(sentence)
printPredict(sen_predicted, sentence)

2
@SBSTransit_Ltd Is there a delay on the NE Line now???
